---
# Begin Analysis #

In [1]:
## ensembl REST API: loci associated with phenotype search

import pandas as pd
import requests, sys, random


random.seed(13)

def getloci(search_typ, term, source):
## 1) NOTE Required to select values for search_typ, term
    # search_typ = "accession" 
        ## ex. "accession" -> phenotype ontology accession; 
        ## ex. "term" -> phenotype ontology term  
    # term = "EFO:0004576"
        ## "fetal hemoglobin"
        ## "EFO:0004576" ## fetal hemoglobin measurement
        ## Orphanet:251380 ## Hereditary persistence of fetal hemoglobin-sickle cell disease syndrome

## 2) NOTE Optional to filter by source; exact match
    if source == 'GWAS':
        source = "NHGRI-EBI GWAS catalog"
        ## ex. "NHGRI-EBI GWAS catalog"; "ClinVar"

    ext = "http://rest.ensembl.org/phenotype/" + str(search_typ) + "/homo_sapiens/" + str(term) + "?source=" + str(source)
    
    r = requests.get(ext, headers={ "Content-Type" : "application/json"})
    
    if not r.ok:
        r.raise_for_status()
        sys.exit()
        ## error checking from ensembl REST API documentation
    
    decoded = r.json()
    df = pd.json_normalize(decoded)
        ## flatten json

    df.rename(columns=(lambda x: x.replace('attributes.', '')), inplace=True)
        ## rename columns for accessibility

    df.dropna(subset=['beta_coefficient'], inplace=True)
        ## drop rows with missing betas
    return df

# Return HPFH variants #

In [2]:
df_hbf = getloci("accession", "EFO:0004576", "GWAS")

df_hbf = df_hbf[df_hbf['beta_coefficient'].str.contains('increase')]

df_hbf.info()
# df_hbf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 1 to 601
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          276 non-null    object
 1   Variation            276 non-null    object
 2   mapped_to_accession  276 non-null    object
 3   location             276 non-null    object
 4   source               276 non-null    object
 5   risk_allele          274 non-null    object
 6   associated_gene      274 non-null    object
 7   external_reference   276 non-null    object
 8   beta_coefficient     276 non-null    object
 9   p_value              276 non-null    object
 10  odds_ratio           0 non-null      object
dtypes: object(11)
memory usage: 25.9+ KB


# Return malaria resistant variants #

In [3]:
df_mal = getloci("accession", "EFO:0001068", "GWAS")

df_mal = df_mal[df_mal['beta_coefficient'].str.contains('decrease')]

# df_mal.head()
df_mal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 4 to 166
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   source               46 non-null     object
 1   location             46 non-null     object
 2   Variation            46 non-null     object
 3   description          46 non-null     object
 4   mapped_to_accession  46 non-null     object
 5   associated_gene      46 non-null     object
 6   external_reference   46 non-null     object
 7   p_value              46 non-null     object
 8   odds_ratio           0 non-null      object
 9   risk_allele          0 non-null      object
 10  beta_coefficient     46 non-null     object
dtypes: object(11)
memory usage: 4.3+ KB


# Return rows with variants in common between datasets #
* # None returned, no variants in common

In [ ]:
pd.merge(df_hbf, df_mal, how='inner', on=['Variation']) 

---------------
# PCA #

----------
### The following code segments were adapted from this author's interpretation during BMI 6332 Summer 2021 of Alistair Miles exploration of Fast PCA in scikit-allel found at https://alimanfoo.github.io/2015/09/28/fast-pca.html

### Genomic data:
    http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.wgs.phase3_shapeit2_mvncall_integrated_v5c.20130502.sites.vcf.gz

### Population cluster sample annotation
    http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20200731.ALL.ped

In [ ]:
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import pandas as pd
import allel

## Data Acquisition
--------
### Cell loads VCF

In [ ]:
callset = allel.read_vcf('ALL.wgs.phase3_shapeit2_mvncall_integrated_v5c.20130502.sites.vcf.gz')

In [ ]:
# gt_zarr = callset['calldata/GT']

## Data Wrangling
--------
## Variants are retained according to specified GRCh37 positions

In [ ]:
pos = allel.SortedIndex(callset['variants/POS'])
loc_region = pos.locate_range(60677655, 60780789)

## Data Wrangling
--------
### 1st cell loads genotype data in array (**gt**) + pulls allele count data into other array -> **ac**
### 2nd cell Filters by MAF cutoff. Transforms array by return the highest allele index for each variant. Transforms each genotype cell into the number of non-reference alleles. 


In [ ]:
gt = allel.GenotypeArray(callset[loc_region])
ac = gt.count_alleles()[:]

In [ ]:
flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 5)

gf = gt.compress(flt, axis=0)
gn = gf.to_n_alt()

## Data Analysis
------
### 1st cell is where SNP is randomly selected for PCA
### 2nd cell is where the PCA is actually performed
        n_components sets how many principal component (PCA fxn outputs) will be produced. 

In [ ]:
n = 100000
vidx = np.random.choice(gn.shape[0], n, replace=False)
vidx.sort()
gnr = gn.take(vidx, axis=0)

In [ ]:
coords1, model1 = allel.pca(gnr, n_components=10, scaler='patterson')

## Data Visualization
----
### 1st cell is where the 2 metadata files are merged to deliver "Continental" data, correlated to "Population"
* "Population" - artificial label roughly defining pop sub group of sample -> "Gujrati Indian from Houston"
* "Continental" - Categorizes "Population" sub groups into continent they emerge from

### 2nd cell takes PCA output data and assigns metadata to each sample. Also assigns variables to be pulled for visualization.

In [ ]:
df_cont = pd.read_csv('continental.txt', delimiter='\t', index_col='Population')

df_samples = pd.read_csv('integrated_call_samples_v3.20200731.ALL.ped.txt', delimiter='\t', index_col='Individual ID')
df_samples = df_samples.join(df_cont, on='Population')

In [ ]:
df_gnu = pd.DataFrame(data=coords1, index=callset['samples'])
pca_samples = df_gnu.join(df_samples)

populations = pca_samples.Population.unique()
continents = pca_samples.Continental.unique()
cont_colours = plt.cm.rainbow(np.linspace(0, 1, len(continents)))
pop_colours = plt.cm.rainbow(np.linspace(0, 1, len(populations)))

## Data Visualization (continued)
----
## Graphical output
* 1st cell returns label and color information for samples and labels explained variance ratio for each principle component
* 2nd cell returns Continental grouping PCA plots

In [ ]:
def plot_pca_coords(coords, model, pc1, pc2, ax, sample_population, pops, pop_colors):
    sns.despine(ax=ax, offset=5)
    x = coords[:, pc1]
    y = coords[:, pc2]
    for pop,color in zip(pops, pop_colors):
        flt = (sample_population == pop)
        ax.plot(x[flt], y[flt], marker='o', linestyle=' ', color=color, 
                label=pop, markersize=6, mec='k', mew=.5)
    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, model.explained_variance_ratio_[pc1]*100))
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, model.explained_variance_ratio_[pc2]*100))

In [ ]:
def fig_pca_cont(axl, ax2, coords, model, title, sample_population = None):
    if sample_population is None:
        sample_population = pca_samples.Continental.values
    plot_pca_coords(coords, model, 0, 1, axl, sample_population, continents, cont_colours)
    plot_pca_coords(coords, model, 2, 3, ax2, sample_population, continents, cont_colours)
    ax2.legend(bbox_to_anchor=(1, 1), loc= 'upper left')
    fig.suptitle(title, y=1.02)
    fig.tight_layout()

fig= plt.figure(figsize=(10, 5))
axl = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

fig_pca_cont(axl, ax2, coords1, model1, 'Figure 1. Continental PCA.')

## LD Pruning
----
### Removal of one SNP from correlated pairs in 5 iterations of a 500 SNP window.
### Variants pruned from 100,000 to only 1,614

In [ ]:
ef ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn

gnu_prune = ld_prune(gnr, size=500, step=200, threshold=.1, n_iter=5) ### NOTE: Uncomment to test

In [ ]:
gnu_prune = gnu_prune[:]
coords2, model2 = allel.pca(gnu_prune, n_components=10, scaler='patterson')
#################################################################################################

def fig_pca_cont_pruned(axl, ax2, coords, model, title, sample_population = None):
    if sample_population is None:
        sample_population = pca_samples.Continental.values
    plot_pca_coords(coords, model, 0, 1, axl, sample_population, continents, cont_colours)
    plot_pca_coords(coords, model, 2, 3, ax2, sample_population, continents, cont_colours)
    ax2.legend(bbox_to_anchor=(1, 1), loc= 'upper left')
    fig.suptitle(title, y=1.02)
    fig.tight_layout()

fig= plt.figure(figsize=(10, 5))
axl = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

####################################


fig_pca_cont_pruned(axl, ax2, coords2, model2, 'Figure 2. Pruned Continental PCA.')